In [1]:
import warnings
warnings.filterwarnings("ignore")

In [283]:
import scripts.data_layer as data_layer

In [5]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import sys
sys.path.append("..")
#import scripts.data_layer as data_layer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Loading data


In [6]:
transactions = pd.read_csv('data_500.csv')

In [8]:
transactions=transactions.groupby(['Customer_id', 'Brand']).size().reset_index(name="Count")

In [9]:
transactions.head(5)

,Customer_id,Brand,Count
0,Customer_0,altar'd state,1
1,Customer_0,ann taylor loft,1
2,Customer_0,armani,1
3,Customer_0,asics,1
4,Customer_0,autozone,2


In [11]:
transactions["Brand"].nunique()  #625 Brands are there

625

In [12]:
Brand_List=list(set(transactions["Brand"]))

In [29]:
enum=enumerate(Brand_List)
brand_dict=dict((j,i) for i,j in enum)

In [31]:
def convertBrandID(x):
    return brand_dict[x]

In [32]:
transactions["products"]=transactions["Brand"].apply(convertBrandID)

In [33]:
transactions.head(10)

,Customer_id,Brand,Count,products
0,Customer_0,altar'd state,1,215
1,Customer_0,ann taylor loft,1,446
2,Customer_0,armani,1,559
3,Customer_0,asics,1,578
4,Customer_0,autozone,2,579
5,Customer_0,avis car rental,1,238
6,Customer_0,best buy,1,284
7,Customer_0,big lots,2,120
8,Customer_0,bmw group,1,305
9,Customer_0,"boost worldwide, inc.",5,95


In [34]:
transactions.columns=["customerId","Brand","purchase_count","productId"]

In [35]:
data=transactions

In [36]:
print(data.shape)  # Which Customer purchased what and in what quantity
data.head()

(105101, 4)


,customerId,Brand,purchase_count,productId
0,Customer_0,altar'd state,1,215
1,Customer_0,ann taylor loft,1,446
2,Customer_0,armani,1,559
3,Customer_0,asics,1,578
4,Customer_0,autozone,2,579


In [37]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [38]:
data_dummy = create_data_dummy(data)

In [39]:
data_dummy.shape

(105101, 5)

In [40]:
data_dummy.head(5) # Added 1 across every Item

,customerId,Brand,purchase_count,productId,purchase_dummy
0,Customer_0,altar'd state,1,215,1
1,Customer_0,ann taylor loft,1,446,1
2,Customer_0,armani,1,559,1
3,Customer_0,asics,1,578,1
4,Customer_0,autozone,2,579,1


In [42]:
data_dummy["productId"].nunique() # There are 625 unique Items

625

In [43]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,...,615,616,617,618,619,620,621,622,623,624
customerId,,,,,,,,,,,,,,,,,,,,,
Customer_0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Customer_1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Customer_10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Customer_100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,...,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Customer_101,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [44]:
df_matrix.shape

(1000, 625)

In [45]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(1000, 625)


productId,0,1,2,3,4,5,6,7,8,9,...,615,616,617,618,619,620,621,622,623,624
customerId,,,,,,,,,,,,,,,,,,,,,
Customer_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Customer_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Customer_10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Customer_100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Customer_101,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [46]:
# create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(99938, 3)


,customerId,productId,scaled_purchase_freq
1,Customer_1,0,0.0
4,Customer_101,0,0.0
14,Customer_110,0,0.0
15,Customer_111,0,0.0
22,Customer_118,0,0.0


#### Define a function for normalizing data

In [47]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

* We can normalize the their purchase history, from 0-1 (with 1 being the most number of purchase for an item and 0 being 0 purchase count for that item).

In [48]:
data_norm.shape

(99938, 3)

In [49]:
data_dummy.shape

(105101, 5)

In [50]:
# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part

norm = tc.SFrame(data_norm)
dummy=tc.SFrame(data_dummy)


In [65]:
norm.head(2)

customerId,productId,scaled_purchase_freq
Customer_1,0,0.0
Customer_101,0,0.0


In [55]:
# variables to define field names
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions['customerId'])
n_rec = 5 # number of items to recommend
n_display = 30

In [56]:
popularity_model = tc.popularity_recommender.create(dummy, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Warning: Ignoring columns Brand, purchase_dummy;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 105101 observations with 1000 users and 625 items.

Data prepared in: 0.156679s

105101 observations to process; with 625 unique items.

In [57]:
len(users_to_recommend)

105101

In [58]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)


recommendations finished on 1000/105101 queries. users per second: 618429

recommendations finished on 2000/105101 queries. users per second: 715820

recommendations finished on 3000/105101 queries. users per second: 742207

recommendations finished on 4000/105101 queries. users per second: 762631

recommendations finished on 5000/105101 queries. users per second: 789640

...

recommendations finished on 101000/105101 queries. users per second: 447612

recommendations finished on 102000/105101 queries. users per second: 445958

recommendations finished on 103000/105101 queries. users per second: 443169

recommendations finished on 104000/105101 queries. users per second: 435256

recommendations finished on 105000/105101 queries. users per second: 433215

# Top 10 trendig restaurants in that city over 1 month

In [59]:
popularity_recomm.print_rows(20)   # These are the recommendations at user level using a Baseline Model

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
| Customer_0 |     66    | 3.0380549682875264 |  1   |
| Customer_0 |    366    | 2.5854748603351956 |  2   |
| Customer_0 |    411    | 2.2359281437125746 |  3   |
| Customer_0 |    417    | 2.153012048192771  |  4   |
| Customer_0 |    466    | 1.9010416666666667 |  5   |
| Customer_0 |     66    | 3.0380549682875264 |  1   |
| Customer_0 |    366    | 2.5854748603351956 |  2   |
| Customer_0 |    411    | 2.2359281437125746 |  3   |
| Customer_0 |    417    | 2.153012048192771  |  4   |
| Customer_0 |    466    | 1.9010416666666667 |  5   |
| Customer_0 |     66    | 3.0380549682875264 |  1   |
| Customer_0 |    366    | 2.5854748603351956 |  2   |
| Customer_0 |    411    | 2.2359281437125746 |  3   |
| Customer_0 |    417    | 2.153012048192771  |  4   |
| Customer_0 |    466    | 1.9010416666666667 |  5   |
| Customer

#### Define a `model` function for model selection

In [71]:
# Since turicreate is very accessible library, we can define a model selection function as below
# The model will decide what should be the similarity criteria like cosine or pearson

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [61]:
customers=pd.DataFrame({"user_id":["Customer_002","Customer_001","Customer_000"]})

In [62]:
customers

,user_id
0,Customer_002
1,Customer_001
2,Customer_000


In [63]:
# variables to define field names
# constant variables include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers["user_id"])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

#### Using purchase dummy

In [76]:
# these variables will change accordingly
name = 'popularity'
target = 'scaled_purchase_freq'
pop_model = model(norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 99938 observations with 1000 users and 427 items.

Data prepared in: 0.08911s

99938 observations to process; with 427 unique items.

+--------------+-----------+---------------------+------+
|  customerId  | productId |        score        | rank |
+--------------+-----------+---------------------+------+
| Customer_002 |     95    | 0.35299999999999765 |  1   |
| Customer_002 |    236    | 0.35161990647962577 |  2   |
| Customer_002 |    105    |  0.3283802491924327 |  3   |
| Customer_002 |    277    |  0.3049999999999998 |  4   |
| Customer_002 |    594    |  0.2859953703703722 |  5   |
| Customer_002 |     87    |  0.2635416666666693 |  6   |
| Customer_002 |    503    |  0.2610011641443535 |  7   |
| Customer_002 |    419    | 0.24971941638608292 |  8   |
| Customer_002 |    131    | 0.24917012448132758 |  9   |
| Customer_002 |    101    |  0.2286814425244198 |  10  |
| Customer_001 |     95    | 0.35299999999999765 |  1   |
| Customer_001 |    236    | 0.35161990647962577 |  2   |
| Customer_001 |    105    |  0.3283802491924327 |  3   |
| Customer_001 |    277    |  0.3049999999999998 |  4   |
| Customer_001

In [79]:
def pred_popular(uid,k):
    recom = pop_model.recommend(users=[uid], k=k)
    recom = recom.to_dataframe() # convertinng sframe to dataframe
    product_list=list(recom["productId"])
    predictions=[brand_dict_reverse[i] for i in product_list]
    return predictions


In [93]:
pred_popular("Customer_002",10) #Same recommendation for All

['boost worldwide, inc.',
 'redbox',
 'valvoline',
 'shell',
 'subway',
 'u-haul',
 'fedex',
 'timberland',
 'valero',
 'interstate batteries']

In [94]:
pred_popular("Customer_001",10) #Same recommendation for All

['boost worldwide, inc.',
 'redbox',
 'valvoline',
 'shell',
 'subway',
 'u-haul',
 'fedex',
 'timberland',
 'valero',
 'interstate batteries']

#### Final Model

In [83]:
final_model = tc.item_similarity_recommender.create(tc.SFrame(norm), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='scaled_purchase_freq', 
                                            similarity_type='pearson')

Preparing data set.

Data has 99938 observations with 1000 users and 427 items.

Data prepared in: 0.078585s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 127.564ms                           | 100              | 427             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.135781s

In [111]:
def pred_results(uid,k):
    recom = final_model.recommend(users=[uid], k=k)
    recom = recom.to_dataframe() # convertinng sframe to dataframe
    product_list=list(recom["productId"])
    predictions=[brand_dict_reverse[i] for i in product_list]
    return predictions
    

In [99]:
pred_results("Customer_0",10) #Different Predictions as per Customers history

['state farm',
 'ultra mobile',
 'jpmorgan chase bank, n.a.',
 'tire rack',
 'wells fargo',
 'goodyear',
 'citgo petroleum corporation',
 "domino's pizza",
 "popeye's chicken",
 'laz parking']

In [95]:
pred_results("Customer_002",10) #Different Predictions as per Customers history

['boost worldwide, inc.',
 'redbox',
 'valvoline',
 'shell',
 'subway',
 'u-haul',
 'fedex',
 'timberland',
 'valero',
 'interstate batteries']

In [112]:
pred_results("Customer_100",10) ##Different Predictions as per customer Customers history

['cricket communications',
 'cvs pharmacy',
 'wells fargo',
 'citgo petroleum corporation',
 'laz parking',
 'timewise',
 'best western inn',
 'hertz',
 'radisson',
 'napa auto parts']

In [110]:
pred_results("Customer_500",10) ##Different Predictions as per customers history

['subway',
 'jpmorgan chase bank, n.a.',
 'cvs pharmacy',
 'best western inn',
 'cummins',
 'radisson',
 'napa auto parts',
 'best western plus',
 "church's chicken",
 'la quinta inn']

In [107]:
pred_results("Deepak",10) ##New Customer trending items

['boost worldwide, inc.',
 'redbox',
 'valvoline',
 'shell',
 'subway',
 'u-haul',
 'fedex',
 'timberland',
 'valero',
 'interstate batteries']

# Saving final_model 

In [102]:
final_model.save("recommender.h5")

In [103]:
pop_model.save("popular_recommender.h5")

In [77]:
brand_dict_reverse={k:v for v,k in brand_dict.items() }

In [105]:
import pickle
with open("brand_dict_reverse.pkl","wb") as rd:
    pickle.dump(brand_dict_reverse,rd)

In [88]:
transactions.head(2)

,customerId,Brand,purchase_count,productId
0,Customer_0,altar'd state,1,215
1,Customer_0,ann taylor loft,1,446


In [89]:
transactions[transactions["customerId"]=="Customer_0"]

,customerId,Brand,purchase_count,productId
0,Customer_0,altar'd state,1,215
1,Customer_0,ann taylor loft,1,446
2,Customer_0,armani,1,559
3,Customer_0,asics,1,578
4,Customer_0,autozone,2,579
...,...,...,...,...
93,Customer_0,verizon wireless,1,108
94,Customer_0,walgreens,4,41
95,Customer_0,wendy's,2,14
96,Customer_0,whirlpool,3,555


In [90]:
pred_results("Customer_0",10)

['state farm',
 'ultra mobile',
 'jpmorgan chase bank, n.a.',
 'tire rack',
 'wells fargo',
 'goodyear',
 'citgo petroleum corporation',
 "domino's pizza",
 "popeye's chicken",
 'laz parking']

In [91]:
transactions[transactions["customerId"]=="Customer_1"]

,customerId,Brand,purchase_count,productId
98,Customer_1,A&W Restaurants,1,316
99,Customer_1,aamco,1,0
100,Customer_1,aaron's,1,361
101,Customer_1,abc supply co.,1,396
102,Customer_1,autozone,3,579
...,...,...,...,...
189,Customer_1,vans shoes,3,99
190,Customer_1,wells fargo,3,466
191,Customer_1,wendy's,1,14
192,Customer_1,whirlpool,1,555


In [92]:
pred_results("Customer_1",10)

['walgreens',
 "peet's coffee & tea",
 'tire rack',
 'pizza hut',
 'cummins',
 'hertz',
 'radisson',
 'napa auto parts',
 'best western plus',
 'at&t store']

In [258]:
transactions[transactions["customerId"]=="Customer_002"]

,customerId,Brand,purchase_count,productId
8,Customer_002,peet's coffee & tea,180,7


In [344]:
pred_results("Customer_002",2)

['mcdonalds', 'subway']

In [260]:

transactions[transactions["customerId"]=="Customer_003"]

,customerId,Brand,purchase_count,productId
9,Customer_003,chevron,30,4
10,Customer_003,domino's pizza,6,9
11,Customer_003,kfc,5,1
12,Customer_003,planet fitness,1,5
13,Customer_003,redbox,44,6
14,Customer_003,starbucks,21,8


In [345]:
pred_results("Customer_003",2)

['mcdonalds', 'subway']

In [262]:
transactions[transactions["customerId"]=="Customer_004"]

,customerId,Brand,purchase_count,productId
15,Customer_004,chevron,26,4
16,Customer_004,domino's pizza,7,9
17,Customer_004,kfc,9,1
18,Customer_004,planet fitness,1,5
19,Customer_004,redbox,42,6
20,Customer_004,starbucks,17,8


In [346]:
pred_results("deepak",3)

['chevron', 'mcdonalds', 'subway']

In [264]:
transactions[transactions["customerId"]=="Customer_005"]

,customerId,Brand,purchase_count,productId
21,Customer_005,chevron,18,4
22,Customer_005,domino's pizza,4,9
23,Customer_005,kfc,3,1
24,Customer_005,planet fitness,4,5
25,Customer_005,redbox,38,6
26,Customer_005,starbucks,16,8


In [318]:
pred_results("Customer_005",5)

['subway', 'mcdonalds', "peet's coffee & tea"]

In [233]:
transactions[transactions["customerId"]=="Customer_006"]

,customerId,Brand,purchase_count,productId
27,Customer_006,chevron,18,4
28,Customer_006,domino's pizza,7,9
29,Customer_006,kfc,4,1
30,Customer_006,planet fitness,1,5
31,Customer_006,redbox,31,6
32,Customer_006,starbucks,11,8


In [319]:

pred_results("Customer_006",2)

['mcdonalds', 'subway']

In [235]:
transactions[transactions["customerId"]=="Customer_007"]

,customerId,Brand,purchase_count,productId
33,Customer_007,chevron,27,4
34,Customer_007,domino's pizza,5,9
35,Customer_007,kfc,3,1
36,Customer_007,planet fitness,2,5
37,Customer_007,redbox,41,6
38,Customer_007,starbucks,22,8


In [320]:
pred_results("Customer_007",2)

['mcdonalds', 'subway']

In [237]:
pred_results("Customer_008",2)

['mcdonalds', 'subway']

In [238]:
pred_results("Customer_009",2)

['mcdonalds', 'subway']

In [239]:
pred_results("Customer_010",2)

['mcdonalds', 'subway']

In [242]:
pred_results(1,2)

['mcdonalds', 'subway']

### Conclusion :  We are able to predict Brands to users based on pearson correlation on scaled purchase frequency